In [1]:
import yaml
import os
import datetime as dt
import pandas as pd
import pytz
import numpy as np
from os.path import (
    abspath,
    dirname,
)


In [8]:
def parse_chaos_attr(chaos_attr, chaos_type):
    chaos_attr_dict = {}
    if chaos_type == 'network':
        action = chaos_attr['action']
        if action == 'loss':
            chaos_attr_dict['Network_loss_rate'] = int(chaos_attr['loss']['loss'])
        elif action == 'delay':
            chaos_attr_dict['Network_delay_latency'] = int(chaos_attr['delay']['latency'].strip('ms'))
        elif action == 'duplicate':
            chaos_attr_dict['Network_duplicate_rate'] = int(chaos_attr['duplicate']['duplicate'])
        elif action == 'corrupt':
            chaos_attr_dict['Network_corrupt_rate'] = int(chaos_attr['corrupt']['corrupt'])
        elif action == 'bandwidth':
            chaos_attr_dict['Network_bandwidth_limit'] = int(chaos_attr['bandwidth']['limit'])
            chaos_attr_dict['Network_bandwidth_buffer'] = int(chaos_attr['bandwidth']['buffer'])
        else:
            pass
    elif chaos_type == 'stress':
        action = list(chaos_attr['stressors'].keys())[0]
        if action == 'cpu':
            chaos_attr_dict['Stress_stressors_cpu_load'] = int(
                chaos_attr['stressors']['cpu']['load'])
        elif action == 'memory':
            chaos_attr_dict['Stress_stressors_memory_size'] = int(
                chaos_attr['stressors']['memory']['size'].strip('%'))
        else:
            print('Unknown action: {}'.format(action))
    elif chaos_type == 'time':
        chaos_attr_dict['Time_timeOffset'] = int(
            chaos_attr['timeOffset'].strip('m'))
    elif chaos_type == 'io':
        action = chaos_attr['action']
        if action == 'latency':
            chaos_attr_dict['IO_delay_time'] = int(chaos_attr['delay'].strip('ms'))
        elif action == 'fault':
            chaos_attr_dict['IO_error_no'] = int(chaos_attr['errno'])
    else:
        pass

    return chaos_attr_dict


In [9]:
# set time
start_time = dt.datetime(2022, 5, 21, 1, 00, 00)+dt.timedelta(hours=8)
end_time = start_time+dt.timedelta(hours=12)-dt.timedelta(seconds=1)
chaos_start_time = dt.datetime(2022, 5, 21, 1, 8, 33)+dt.timedelta(hours=8)
chaos_duration=dt.timedelta(seconds=60)
suspend_duration=dt.timedelta(minutes=10)

event_type_list = {'normal': 0}
chaos_df = pd.DataFrame(
    columns=[
        'timestamp', 'event_type', 'IO_delay_time', 'IO_error_no',
        'Network_loss_rate', 'Network_delay_latency', 
        'Network_duplicate_rate', 'Network_corrupt_rate',
        'Network_bandwidth_limit', 'Network_bandwidth_buffer',
        'Stress_stressors_cpu_load', 'Stress_stressors_memory_size',
    ])

# set chaos_df timestamp
chaos_df['timestamp'] = pd.date_range(
    start=start_time, end=end_time, freq='1s', tz='Asia/Shanghai')
chaos_df['timestamp'] = chaos_df['timestamp'].apply(
    lambda x: x.timestamp())

# fill all other columns with 0
chaos_df.fillna(0, inplace=True)


In [10]:
with open(os.path.join(dirname(dirname(os.getcwd())), "chaos_sql_0.yaml"), "r") as stream:
    try:
        wf_data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)


In [11]:
current_time = chaos_start_time
for event in wf_data["spec"]["templates"][1:]:
    if event["templateType"] == "Suspend":
        current_time += suspend_duration
    else:
        chaos_type = event["templateType"].split("Chaos")[0]
        chaos_type = chaos_type.lower()

        type_name = '_'.join(event["name"].split("-")[:2]) if event["name"].split("-")[
            0] != 'time' else 'time'
        if event_type_list.get(type_name) is None:
            event_type_list[type_name] = len(event_type_list)
        # set 30s interval after current_time on chaos_df column as event_type
        chaos_df.loc[(chaos_df['timestamp'] >= current_time.timestamp()) &
                     (chaos_df['timestamp'] <=
                      (current_time + chaos_duration).timestamp()),
                     'event_type'] = event_type_list[type_name]

        chaos_attr_dict = parse_chaos_attr(
            event[chaos_type+'Chaos'], chaos_type)
        for c in chaos_attr_dict:
            chaos_df.loc[(chaos_df['timestamp'] >= current_time.timestamp()) & (
                chaos_df['timestamp'] <= (current_time+dt.timedelta(seconds=30)
                                          ).timestamp()), c] = chaos_attr_dict[c]
        current_time += dt.timedelta(seconds=30)


In [12]:
chaos_df.to_csv(os.path.join(
    dirname(dirname(os.getcwd())), "output", 'mysql', 'chaos_info.csv'), index=False)


In [13]:
event_type_list

{'normal': 0,
 'io_latency': 1,
 'io_fault': 2,
 'network_part': 3,
 'network_loss': 4,
 'network_delay': 5,
 'network_dup': 6,
 'network_corr': 7,
 'network_band': 8,
 'cpu_stress': 9,
 'mem_stress': 10}

In [ ]:
for event in wf_data["spec"]["templates"][1:]:
    if event["name"] == "time-tikv-all-p15":
        chaos_type = event["templateType"].split("Chaos")[0]
        chaos_type = chaos_type.lower()
        chaos_attr_dict = parse_chaos_attr(
            event[chaos_type+'Chaos'], chaos_type)
        print(event[chaos_type+'Chaos'])
        print('======================================')
        print(chaos_attr_dict)


In [38]:
chaos_df=pd.read_csv(os.path.join(os.pardir, "output", 'chaos.csv'))
data_df=chaos_df.drop(['timestamp'],axis=1)

In [39]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1)
pca.fit(data_df)

pca_res = pca.transform(data_df)
chaos_disc={}
chaos_state=[]

for c in pca_res.reshape(-1):
    if chaos_disc.get(c) is None:
        chaos_disc[c]=len(chaos_disc)
    chaos_state.append(chaos_disc[c])

chaos_state_df=pd.DataFrame(chaos_state, columns=['chaos_state'])
chaos_state_df["timestamp"]=chaos_df["timestamp"]
chaos_state_df.to_csv(os.path.join(os.pardir, "output", 'chaos_state.csv'), index=False)

In [40]:
chaos_disc


{-11.888919722132187: 0,
 -11.887834029712968: 1,
 -11.887834681382989: 2,
 -11.88783422146851: 3,
 -11.887651101218378: 4,
 -11.8868403191316: 5,
 -11.886840970801622: 6,
 -11.886840510887144: 7,
 -11.886657390637009: 8,
 -11.885846608550231: 9,
 -11.885846531834378: 10,
 -11.885847260220253: 11,
 -11.885846800305774: 12,
 -11.88566368005564: 13,
 -11.884852897968862: 14,
 -11.88485282125301: 15,
 -11.884853549638883: 16,
 -11.884853089724405: 17,
 -11.88466996947427: 18,
 -11.883859187387495: 19,
 -11.88385911067164: 20,
 -11.883859839057514: 21,
 -11.883859379143036: 22,
 -11.883676258892903: 23,
 -11.882498507919983: 24,
 -11.881734848299033: 25,
 -11.88196489925945: 26,
 -11.88219495021987: 27,
 -11.882425001180287: 28,
 -11.882655052140704: 29,
 -11.880649119419573: 30,
 -11.881201250068887: 31,
 -11.881891413380528: 32,
 -11.88327174000381: 33,
 -11.879747388655636: 34,
 -11.879977401135394: 35,
 -11.880207413615151: 36,
 -11.88043742609491: 37,
 -11.880667438574667: 38,
 -11.87